Поэкспериментировать с моделью генерации текста, попробовать модель на словах, посмотреть что отработает лучше

In [156]:
import tensorflow as tf
import string
import numpy as np
import os
import time

In [157]:
path_to_file = 'evgenyi_onegin.txt'
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(text[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [158]:
data = text.split('\n')
data = data[5:]
data[0]

'                        Не мысля гордый свет забавить,'

In [159]:
#всего строк в романе
len(data)

6501

In [160]:
#теперь мы собираемся соединить все строки и создать длинную строку, состоящую из данных в непрерывном формате
data = " ".join(data)

In [161]:
#Мы собираемся создать функцию clean_text (), чтобы удалить все знаки препинания и специальные символы из данных
def clean_text(doc):
  tokens = doc.split()
  table = str.maketrans('', '', string.punctuation)
  tokens = [w.translate(table) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  tokens = [word.lower() for word in tokens]
  return tokens

tokens = clean_text(data)
print(tokens[:50])

['не', 'мысля', 'гордый', 'свет', 'забавить', 'вниманье', 'дружбы', 'возлюбя', 'хотел', 'бы', 'я', 'тебе', 'представить', 'залог', 'достойнее', 'тебя', 'достойнее', 'души', 'прекрасной', 'святой', 'исполненной', 'мечты', 'поэзии', 'живой', 'и', 'ясной', 'высоких', 'дум', 'и', 'простоты', 'но', 'так', 'и', 'быть', 'рукой', 'пристрастной', 'прими', 'собранье', 'пестрых', 'глав', 'полусмешных', 'полупечальных', 'простонародных', 'идеальных', 'небрежный', 'плод', 'моих', 'забав', 'бессонниц', 'легких']


In [162]:
#всего токенов
len(tokens)

23006

In [163]:
#всего уникальных токенов
len(set(tokens))

8515

### train and target

мы собираемся использовать набор предыдущих слов, чтобы предсказать следующее слово в предложении. Чтобы быть точным, мы собираемся использовать набор из 50 слов, чтобы предсказать 51-е слово. Следовательно, мы собираемся разделить наши данные на блоки по 51 слову и, наконец, отделим последнее слово от каждой строки

In [164]:
length = 50 + 1
lines = []

for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    lines.append(line)

print(len(lines))

22955


In [165]:
lines[:5]

['не мысля гордый свет забавить вниманье дружбы возлюбя хотел бы я тебе представить залог достойнее тебя достойнее души прекрасной святой исполненной мечты поэзии живой и ясной высоких дум и простоты но так и быть рукой пристрастной прими собранье пестрых глав полусмешных полупечальных простонародных идеальных небрежный плод моих забав бессонниц легких вдохновений',
 'мысля гордый свет забавить вниманье дружбы возлюбя хотел бы я тебе представить залог достойнее тебя достойнее души прекрасной святой исполненной мечты поэзии живой и ясной высоких дум и простоты но так и быть рукой пристрастной прими собранье пестрых глав полусмешных полупечальных простонародных идеальных небрежный плод моих забав бессонниц легких вдохновений незрелых',
 'гордый свет забавить вниманье дружбы возлюбя хотел бы я тебе представить залог достойнее тебя достойнее души прекрасной святой исполненной мечты поэзии живой и ясной высоких дум и простоты но так и быть рукой пристрастной прими собранье пестрых глав полу

In [166]:
#строка содержит 51 слово
lines[0]

'не мысля гордый свет забавить вниманье дружбы возлюбя хотел бы я тебе представить залог достойнее тебя достойнее души прекрасной святой исполненной мечты поэзии живой и ясной высоких дум и простоты но так и быть рукой пристрастной прими собранье пестрых глав полусмешных полупечальных простонародных идеальных небрежный плод моих забав бессонниц легких вдохновений'

In [167]:
#51 слово будем использовать для предсказания
tokens[50]

'вдохновений'

In [168]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding, Input
from tensorflow.keras.preprocessing.sequence import pad_sequences

Мы собираемся создать уникальный числовой токен для каждого уникального слова в наборе данных. fit_on_texts () обновляет внутренний словарь на основе списка текстов. text_to_sequences () преобразует каждый текст в текстах в последовательность целых чисел.

In [169]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

sequence содержит список целочисленных значений, созданный токенизатором. Каждая строка в последовательностях содержит 51 слово. Теперь мы разделим каждую строку так, чтобы первые 50 слов были в X, а последнее слово было в y.

In [170]:
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:,-1]
X[0]

array([   3, 8515,  891,  119, 8514, 2565,  890, 8513,  507,   58,    6,
        231,  889,  888, 8504,  173, 8504,  104, 1357, 1356, 8509,  144,
       1355,  352,    1, 8507, 2562,  643,    1, 1354,    8,   18,    1,
         50,  350, 8503, 2561,  641, 2560, 8501, 8500, 8499, 8497, 8496,
       2559, 1353,  348,  639, 8494, 8493])

In [171]:
y[0]

2566

Vocab_size содержит все уникальные слова в наборе данных. tokenizer.word_index отображает отображение каждого уникального слова на его числовой эквивалент. Следовательно, len () для tokenizer.word_index дает словарный_размер.

In [172]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

8516

In [173]:
y

array([2566, 2567,    1, ...,    7,  646,  647])

In [174]:
y = to_categorical(y, num_classes=vocab_size)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [175]:
seq_length = X.shape[1]
seq_length

50

In [183]:
# The embedding dimension
embedding_dim = 300

# Number of RNN units
rnn_units = 512

# Batch size
BATCH_SIZE = 64

In [190]:
X.shape[0]

22955

In [194]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size, seq_length):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim
                                  #,batch_input_shape=[batch_size, embedding_dim]
                                 ),
                                 
        tf.keras.layers.GRU(rnn_units,
                            batch_input_shape=(batch_size, X.shape[0], 1),
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            batch_input_shape=(batch_size, X.shape[0], 1),
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                             batch_input_shape=(batch_size, X.shape[0], 1),
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size, activation='softmax')
    ])
    return model

In [195]:
model = build_model(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE,
    seq_length = seq_length)

ValueError: If a RNN is stateful, it needs to know its batch size. Specify the batch size of your input tensors: 
- If using a Sequential model, specify the batch size by passing a `batch_input_shape` argument to your first layer.
- If using the functional API, specify the batch size by passing a `batch_shape` argument to your Input layer.

In [186]:
model.summary()

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_29 (Embedding)     (64, 300, 300)            2554800   
_________________________________________________________________
gru_72 (GRU)                 (64, 300, 512)            1250304   
_________________________________________________________________
gru_73 (GRU)                 (64, 300, 512)            1575936   
_________________________________________________________________
gru_74 (GRU)                 (64, 300, 512)            1575936   
_________________________________________________________________
dense_23 (Dense)             (64, 300, 8516)           4368708   
Total params: 11,325,684
Trainable params: 11,325,684
Non-trainable params: 0
_________________________________________________________________


## Train the model

In [187]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [188]:
model.fit(X, y, batch_size = BATCH_SIZE, epochs = 100)

Epoch 1/100


ValueError: in user code:

    c:\users\sych_\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    c:\users\sych_\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\sych_\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\sych_\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\sych_\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replic

generate_text_seq () генерирует n_words количество слов после заданного seed_text. Мы собираемся предварительно обработать seed_text перед предсказанием. Мы собираемся закодировать seed_text, используя ту же кодировку, которая использовалась для кодирования обучающих данных. Затем мы собираемся преобразовать seed_text в 50 слов с помощью pad_sequences (). Теперь будем делать прогнозы с помощью model.predict_classes (). После этого мы будем искать слово в токенизаторе, используя индекс в y_predict. Наконец, мы добавим предсказанное слово к seed_text и text и повторим процесс.

In [ ]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
  text = []

  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')

    y_predict = model.predict_classes(encoded)

    predicted_word = ''
    for word, index in tokenizer.word_index.items():
      if index == y_predict:
        predicted_word = word
        break
    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)
  return ' '.join(text)